In [ ]:
#!/bin/bash
# https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri/data
# !kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri
# !unzip "./brain-tumor-classification-mri.zip"


In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, precision_score
import numpy as np
import torchvision.transforms as v2 
import albumentations as A
import mlflow
from aux import ImagesMRIDataset, split_for_cross_validation, get_training_testing_data, split_traing_data 
mlflow.set_tracking_uri("http://localhost:5000")

class EarlyStopping():
    def __init__(self, path : str, patience=5, threshold=1e-4):
        self.patience = patience
        self.threshold = threshold
        self.min_loss = 10000
        self.steps_till_stop = 0
        self.path = path

    def continue_training(self, model, loss):
        if(loss < self.min_loss - self.threshold):
            self.min_loss = loss
            self.steps_till_stop = 0
            torch.save(model.state_dict(), self.path)
            return True
        if (loss >= self.min_loss - self.threshold):
            self.steps_till_stop += 1
            if (self.steps_till_stop == self.patience): return False
        return True
    
    def load_model(self, model):
        model.load_state_dict(torch.load(self.path, weights_only=True))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        return model


In [ ]:
def training_loop(model, criterion, optimizer, dataloader : DataLoader):
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   current_training_loss = 0
   all_train_labels, all_train_preds = [], []
   model.train()
   for idx, (images, labels) in enumerate(dataloader):
      images, labels = images.to(device), labels.to(device)
      output = model(images)
      output = output.to(device)
      loss = criterion(output, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      current_training_loss += loss.item()
      all_train_preds.extend(output.argmax(dim=1).cpu().numpy())
      all_train_labels.extend(labels.cpu().numpy())
   loss = current_training_loss / len(dataloader)
   acc = round(accuracy_score(all_train_preds, all_train_labels), 3)
   return loss, acc

def validation_loop(model, criterion, dataloader : DataLoader):
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   all_val_labels, all_val_preds = [], [],
   current_validation_loss = 0
   model.eval()
   with torch.no_grad():
      for idx, (images, labels) in enumerate(dataloader):
         images, labels = images.to(device), labels.to(device)
         output = model(images)
         output = output.to(device)
         loss = criterion(output, labels)
         current_validation_loss += loss.item()
         all_val_labels.extend(labels.cpu().numpy())
         all_val_preds.extend(output.argmax(dim=1).cpu().numpy())
   
   loss = current_validation_loss / len(dataloader)
   acc =  round(accuracy_score(all_val_labels, all_val_preds), 3)
   return loss, acc,   

def train_model(parameters, model, data):
  earlyStopping : EarlyStopping = parameters["early_stopping"]
  current_acc = 0.0
  for i in range(parameters["epochs"]):
      tloss, tacc = training_loop(model, parameters["criterion"], parameters["optimizer"], data["train"])
      mlflow.log_metrics(
      metrics={
            "train_loss": tloss,
            "train_accuracy": tacc
         },
         step=i
      )
      vloss, vacc = validation_loop(model, parameters["criterion"], data["validation"])
      mlflow.log_metrics(
      metrics={
          "validation_loss": vloss,
          "validation_accuracy": vacc,
          },
          step=i
      )
      if (earlyStopping != None and not earlyStopping.continue_training(model, vloss)):
         print("Acuratetea nu a crescut de ceva vreme, a intervenit early stopping")
         break
      if (vacc > current_acc): 
         current_acc = vacc
         print(f"Acuratetea maxima {vacc}, la epoca {i}, loss de {vloss}")
      else:
         print(f"acuratete {vacc}, epoca {i}, loss {vloss}")

In [37]:
class Net(nn.Module):
    def __init__(self,  width : int, expansion : int):
        super().__init__()
        self.relu = nn.ReLU()
        self.width1 = width
        self.conv1 = nn.Conv2d(3, self.width1, kernel_size=(3, 3))
        self.bn1 = nn.BatchNorm2d(self.width1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width2 = self.width1 * expansion
        self.conv2 = nn.Conv2d(self.width1, self.width2, kernel_size=(3, 3))
        self.bn2 = nn.BatchNorm2d(self.width2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width3 = self.width2 * expansion
        self.conv3 = nn.Conv2d(self.width2, self.width3, kernel_size=(5, 5))
        self.bn3 = nn.BatchNorm2d(self.width3)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width4 = self.width3 * expansion
        self.conv4 = nn.Conv2d(self.width3, self.width4, kernel_size=(3, 3))
        self.bn4 = nn.BatchNorm2d(self.width4)
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width5 = self.width4 * expansion
        self.conv5 = nn.Conv2d(self.width4, self.width5, kernel_size=(3, 3))
        self.bn5 = nn.BatchNorm2d(self.width5)

        # (1024, 12) -> (1024, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classification_layer = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(self.width5, self.width4),
            nn.Dropout(p=0.5),
            nn.Linear(self.width4, self.width3),
            nn.Dropout(p=0.5),
            nn.Linear(self.width3, 4),
            # nn.Softmax(dim=1)
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        x = self.maxpool1(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool2(self.relu(self.bn2(self.conv2(x))))
        x = self.maxpool3(self.relu(self.bn3(self.conv3(x))))
        x = self.maxpool4(self.relu(self.bn4(self.conv4(x))))
        x = self.avgpool(self.relu(self.bn5(self.conv5(x))))
        return self.classification_layer(x)

In [ ]:
def create_cross_validation_data(parameters, data_chunks, test_data, current : int):
    training_data = []
    for idx, chunk in enumerate(data_chunks):
        if (idx == current):
            continue
        training_data += chunk
    return {
        "train": DataLoader(ImagesMRIDataset(training_data, transformations=parameters["train_transform"]), batch_size=parameters["batch_size"], shuffle=True, drop_last=True),
        "validation": DataLoader(ImagesMRIDataset(data_chunks[current], transformations=parameters["test_transform"]), batch_size=parameters["batch_size"], shuffle=True),
        "test": DataLoader(ImagesMRIDataset(test_data, transformations=parameters["test_transform"]), batch_size=parameters["batch_size"], shuffle=True)
    }

def compute_cross_validation(parameters, K, data_chunks, test_info):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for idx in range(0, K):
        if (idx > 0): return
        print(f"Antrenare fold {idx}:")
        with mlflow.start_run(run_name=f"fold_{idx}") as run:
            model = Net(width=parameters["width"], expansion=parameters["expansion"])
            parameters["optimizer"] = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
            model = model.to(device)
            mlflow.log_params(parameters)
            data = create_cross_validation_data(parameters, data_chunks, test_info, idx)
            train_model(parameters, model, data)
            if (parameters["early_stopping"]  != None):
                best_model = Net(width=parameters["width"], expansion=parameters["expansion"])
                parameters["early_stopping"].load_model(best_model)
            else:
                best_model = model 
            loss, acc = validation_loop(best_model, parameters["criterion"], data["test"])
            mlflow.log_metrics(
                metrics={
              "test_loss": loss,
              "test_accuracy": acc,
              }
            )
            print(f"pentru test: acuratete {acc}, pierderea este {loss}")

In [ ]:
from collections import Counter

K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
# training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

training_no_transforms = v2.Compose([
    v2.ToTensor(),
    v2.Resize((100, 100))
])

# EarlyStopping(path="./aici.pth", patience=5)
parameters = {
        "batch_size": 64,
        "epochs" : 3,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": training_no_transforms,
        "test_transform": training_no_transforms,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    }

# task 1 - fara augmentari, duplicarea exemplelor din clasele suplimentare
mlflow.set_experiment(experiment_name="Task1")
compute_cross_validation(parameters, K, data_chunks, test_info)

Antrenare fold 0:
Acuratetea maxima 0.517, la epoca 0, loss de 1.1574879126115278
Acuratetea maxima 0.593, la epoca 1, loss de 1.0238515680486506
Acuratetea maxima 0.605, la epoca 2, loss de 0.9415839368646796
True
pentru test: acuratete 0.307, pierderea este 1.5513173852648054
🏃 View run fold_0 at: http://localhost:5000/#/experiments/697423403428355787/runs/15546cf691864f7caec96d1e26f0e7d7
🧪 View experiment at: http://localhost:5000/#/experiments/697423403428355787


In [34]:
SIZE = (100, 100)
train_transform = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),   
    A.HorizontalFlip(p = 0.5),
    # A.CLAHE(clip_limit=5.0, tile_grid_size=(8, 8), p=1.0), 
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8), 
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

test_transformations = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

def compute_train_transformations(image):
    image = train_transform(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)
    
def compute_test_transformations(image):
    image = test_transformations(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)


In [38]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
data_chunks = split_for_cross_validation(training_info, K)

# task 2 - cu augmentari, cu duplicarea exemplelor din clasele suplimentare
parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": compute_train_transformations,
        "test_transform": compute_test_transformations,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    }

mlflow.set_experiment(experiment_name="Task2")
compute_cross_validation(parameters, K, data_chunks, test_info)

Antrenare fold 0:
Acuratetea maxima 0.25, la epoca 0, loss de 1.395501732826233
Acuratetea maxima 0.504, la epoca 1, loss de 1.168650507926941
Acuratetea maxima 0.618, la epoca 2, loss de 0.9428816636403402
Acuratetea maxima 0.694, la epoca 3, loss de 0.7541070481141409
Acuratetea maxima 0.725, la epoca 4, loss de 0.71403240164121
Acuratetea maxima 0.75, la epoca 5, loss de 0.6779457628726959
acuratete 0.735, epoca 6, loss 0.6409063637256622
Acuratetea maxima 0.79, la epoca 7, loss de 0.567603588104248
acuratete 0.774, epoca 8, loss 0.6490016480286916
acuratete 0.748, epoca 9, loss 0.6551865041255951
Acuratetea maxima 0.808, la epoca 10, loss de 0.47614013652006787
Acuratetea maxima 0.816, la epoca 11, loss de 0.44285225371519726
acuratete 0.742, epoca 12, loss 0.656170129776001
Acuratetea maxima 0.827, la epoca 13, loss de 0.46151144802570343
acuratete 0.805, epoca 14, loss 0.4709816525379817
Acuratetea maxima 0.837, la epoca 15, loss de 0.4279339810212453
Acuratetea maxima 0.86, la e

In [27]:
K = 5
BALANCED = False
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

def compute_class_weights(data_stream):
    labels = [label for _, label in data_stream]
    label_counts = Counter(labels)
    total_samples = sum(label_counts.values())
    class_weights = {label: total_samples / count for label, count in label_counts.items()}
    max_label = max(label_counts.keys())
    weights_list = [class_weights.get(i, 0.0) for i in range(max_label + 1)]
    weights = torch.tensor(weights_list, dtype=torch.float)
    print(weights)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    weights = weights.to(device)
    return weights

parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": compute_train_transformations,
        "test_transform": compute_test_transformations,
        "criterion": torch.nn.CrossEntropyLoss(weight=compute_class_weights(training_info)),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    } 

# Task 2 cu augmentari - cu functia de loss avand weights
mlflow.set_experiment(experiment_name="Task2 augmentari")
compute_cross_validation(parameters, K, data_chunks, test_info)

tensor([3.4758, 3.4916, 7.2595, 3.4705])
Antrenare fold 0:
Acuratetea maxima 0.165, la epoca 0, loss de 1.386326789855957
Acuratetea maxima 0.286, la epoca 1, loss de 1.4179309010505676
acuratete 0.286, epoca 2, loss 1.4051107466220856
acuratete 0.286, epoca 3, loss 1.5790435075759888
Acuratetea maxima 0.332, la epoca 4, loss de 1.3863252103328705
Acuratetea maxima 0.477, la epoca 5, loss de 1.1966154277324677
acuratete 0.306, epoca 6, loss 1.6896510124206543
acuratete 0.364, epoca 7, loss 1.4237808883190155
Acuratetea maxima 0.547, la epoca 8, loss de 1.1571098864078522
acuratete 0.542, epoca 9, loss 1.1488657295703888
acuratete 0.531, epoca 10, loss 1.2210025787353516
Acuratetea maxima 0.67, la epoca 11, loss de 0.913056418299675
acuratete 0.67, epoca 12, loss 0.9308577924966812
Acuratetea maxima 0.688, la epoca 13, loss de 0.8520222008228302
acuratete 0.547, epoca 14, loss 1.1130298674106598
acuratete 0.675, epoca 15, loss 0.8412214368581772
acuratete 0.61, epoca 16, loss 0.94400167

In [ ]:
from monai.transforms import Resize, Rand2DElastic, RandAdjustContrast, RandRotate, ScaleIntensity, Compose
from monai.transforms import RandGaussianNoise, RandAffine, RandStdShiftIntensity
from monai.transforms import RandGaussianSmooth, ThresholdIntensity, RandAxisFlip
from monai.transforms import RandSpatialCrop, RandGaussianSharpen, RandHistogramShift, RandShiftIntensity
import cv2

first_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianSharpen(),
    RandAxisFlip(prob=0.3),
    RandSpatialCrop(roi_size=SIZE, random_center=True, random_size=False),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    RandAdjustContrast(prob=0.8),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

second_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    Rand2DElastic(spacing=(5, 10), magnitude_range=(1, 2), prob=0.5, rotate_range=(0, 0), shear_range=(0.02, 0.02),
                    translate_range=(10, 10), scale_range=(0.1, 0.1), spatial_size=SIZE, mode="bilinear", padding_mode="reflection"),
    RandHistogramShift(prob=0.8),
    ThresholdIntensity(threshold=20),
    RandGaussianSmooth(sigma_x=(0.5, 1.5), sigma_y=(0.5, 1.5), prob=0.8),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    ScaleIntensity(minv=0.0, maxv=1.0)
])


third_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianNoise(mean=0.0, std=0.1, prob=0.8),
    RandShiftIntensity(offsets=(-20, 20), prob=0.3, safe=True),
    RandAxisFlip(prob=0.3),
    RandStdShiftIntensity(factors=0.8),
    RandAffine(prob=0.2),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

test_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

def create_test_monai(img):
    img = np.transpose(img, (2, 0, 1))
    return test_augmentation_pipeline(img)

def create_first_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return first_augmentation_pipeline(img)


def create_second_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return second_augmentation_pipeline(img)


def create_third_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return third_augmentation_pipeline(img)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": create_first_monai_transform,
        "test_transform": create_test_monai,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    } 

mlflow.set_experiment(experiment_name="Task3 set1 transformari")
compute_cross_validation(parameters, K, data_chunks, test_info)

In [ ]:
parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": create_second_monai_transform,
        "test_transform": create_test_monai,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    } 
mlflow.set_experiment(experiment_name="Task3 set2 transformari")
compute_cross_validation(parameters, K, data_chunks, test_info)

In [ ]:
parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": create_third_monai_transform,
        "test_transform": create_test_monai,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    } 
mlflow.set_experiment(experiment_name="Task3 set3 transformari")
compute_cross_validation(parameters, K, data_chunks, test_info)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
parameters = {
        "batch_size": 128,
        "epochs" : 20,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": compute_train_transformations,
        "test_transform": compute_test_transformations,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": EarlyStopping("./aici.pth", 5)
    } 
mlflow.set_experiment(experiment_name="Task4 early stopper strategy")
compute_cross_validation(parameters, K, data_chunks, test_info)

NameError: name 'create_third_monai_transform' is not defined

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
current_parameters = {
    "batch_size": 50,
    "epochs" : 25,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.6,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

def compute_functions_with_lr_scheduler(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.CrossEntropyLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"]), None


task4_lr_scheduler_test, task4_lr_scheduler_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_first_monai_transform, create_test_monai, compute_functions_with_lr_scheduler)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
parameters = {
    "batch_size": 16,
    "epochs" : 50,
    "lr": 1e-2,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.6,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

# 72 maxim
def compute_ablatiations(model : Net, parameters):
    # optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    optimizer = torch.optim.SGD(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"], momentum=0.7)
    return torch.nn.BCEWithLogitsLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"]), EarlyStopping("./aici.pth", patience=parameters["early_stopping_patience"])

SIZE=(100, 100)
ablatiation_augmentation = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianSharpen(),
    RandAxisFlip(prob=0.3),
    RandSpatialCrop(roi_size=SIZE, random_center=True, random_size=False),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    RandAdjustContrast(prob=0.8),
    ScaleIntensity(minv=0.0, maxv=1.0)
])


def create_ablatiation_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return ablatiation_augmentation(img)


test_augmentation_ablatiation = Compose([
    Resize(spatial_size= SIZE),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

def create_ablatiation_test_monai(img):
    img = np.transpose(img, (2, 0, 1))
    return test_augmentation_ablatiation(img)

info_test = {"precision": [], "recall": [], "f1Score" : [], "accuracy" : []}
for i in range(K):
    if (i != 1):
        continue
    model = Net(width=parameters["width"], expansion=parameters["expansion"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion, optimizer, lr_scheduler, early_stopping = compute_ablatiations(model, parameters)
    data = create_cross_validation_data(data_chunks, test_info, i, create_ablatiation_transform, create_ablatiation_test_monai, parameters["epochs"])
    training_loss, validation_loss, training_accuracy, validation_accuracy = train_model(model, parameters["epochs"],
            data, criterion, optimizer, lr_scheduler, early_stopping)
    if (early_stopping != None):
        best_model = Net(width=parameters["width"], expansion=parameters["expansion"])
        early_stopping.load_model(best_model)
    else:
        best_model = model
    precision, recall, f1, acc, mat = test_model(best_model, data["validation"], criterion)
    precision, recall, f1, acc, mat = test_model(best_model, data["test"], criterion)